In [184]:
import os
import pandas as pd
import numpy as np
from operator import itemgetter


class Filtrado:
    def __init__(self,hora_inicial=6, hora_final=17):
        # self.nombre_analizador = Analizador
        self.hora_inicial = hora_inicial
        self.hora_final = hora_final   
    
    def Archivos(self,Ruta):
        """
        Esta función recibe una ruta y retorna una lista de rutas completas de archivos en el directorio especificado, excluyendo '.DS_Store' si está presente.

        Args:
            Ruta (str): Ruta del directorio.

        Returns:
            list: Lista de rutas completas de archivos en el directorio.
        """
        archivos = [archivo for archivo in os.listdir(Ruta) if not archivo.startswith('.DS_Store')]
        rutas_completas = [os.path.join(Ruta, archivo) for archivo in archivos]
        return rutas_completas

    def Lectura(self,list_rutas,):
        """
        Lee un archivo CSV o Excel y devuelve un DataFrame.

        Parámetros:
        archivo (str): Ruta del archivo a leer.

        Retorna:
        DataFrame: DataFrame con los datos del archivo leído.

        Errores:
        ValueError: Si el formato del archivo no es compatible.
        IOError: Si ocurre un error al leer el archivo.
        """
        list_df = []
        for ruta in list_rutas:
            try:
                if ruta.endswith(".csv"):
                    df = pd.read_csv(ruta )
                elif ruta.endswith(".xlsx") or ruta.endswith(".xls"):
                    
                    dataframes_por_hoja = pd.read_excel(ruta, sheet_name=None)
                    list_df.append(pd.concat(dataframes_por_hoja.values(), ignore_index=True))
                else:
                    raise ValueError("Formato de archivo no compatible.")
                
            except pd.errors.EmptyDataError:
                return {"error": "El archivo está vacío."}
            except IOError as e:
                return {"error": f"Error al leer el archivo '{ruta}': {e}"}
        return pd.concat(list_df, ignore_index=True)
    
    def Arreglo(self, df,nombre_analizador):
        """
        Filtra y procesa un DataFrame según los criterios establecidos.
        Borrar elementos duplicados y filtrar por analizador y rango de horas.

        Parámetros:
        - df: DataFrame a filtrar y procesar.

        Retorna:
        - df_filtrado: DataFrame filtrado y procesado.
        """
        df = df[df['analizador'] == nombre_analizador].reset_index(drop=True)
        df["datetime"] = pd.to_datetime(df['datetime'])
        df.insert(1, "Fechas", pd.to_datetime(df["datetime"].dt.strftime('%Y-%m-%d %H:%M')))
        del df["datetime"]
        df2 = df.drop_duplicates(subset=['Fechas'])
        df_filtrado = df2[df2['Fechas'].dt.hour.between(self.hora_inicial, self.hora_final)]
        df_filtrado = df_filtrado.reset_index(drop=True)
        return df_filtrado
    
    def Series_tiempo(self, df):
        """
        Calcula la serie de tiempo promedio de la columna 'Pmax' en base a los valores de fecha y hora del DataFrame proporcionado.

        Parámetros:
        - df: DataFrame que contiene la columna 'Fechas' y 'Pmax'.

        Retorna:
        - numpy.array: Un arreglo numpy que contiene los valores promedio de 'Pmax' agrupados por mes, hora y minuto repetidos por mes
        """
        df["Mes"] = df["Fechas"].dt.month
        df["Hora"] = df["Fechas"].dt.hour
        df["Minuto"] = df["Fechas"].dt.minute
        lista = []
        df_filtrado_minutal_prom = df.groupby(["Mes", "Hora", "Minuto"])['Pmax'].mean()
        for i in range(len(df["Fechas"])):
            lista.append(df_filtrado_minutal_prom[(df["Fechas"][i].month, df["Fechas"][i].hour, df["Fechas"][i].minute)],)
        
        return np.array(lista)
    
    def Procesamiento(self, df,nombre_analizador):
        """
        Realiza el procesamiento de un DataFrame dado.
        Le agrega las fechas faltantes de valores faltantes en la columna 'Pmax' de las horas de nuestro interes.

        Parameters:
        - df: DataFrame
            El DataFrame que se desea procesar.

        Returns:
        - df_filtrado: DataFrame
            El DataFrame procesado.
        """
        df_filtrado = self.Arreglo(df,nombre_analizador)
        U = pd.Timestamp(year=df_filtrado["Fechas"].iloc[0].year, month=1, day=1, hour=6)

        fechas_unir = pd.date_range(start=U, end=df_filtrado["Fechas"].iloc[-1], freq='1min')
        data_fechas = pd.DataFrame(fechas_unir, columns=["Fechas"])
        data_fechas2 = data_fechas.loc[data_fechas['Fechas'].dt.hour.between(self.hora_inicial, self.hora_final)]
        df_filtrado = pd.merge(df_filtrado, data_fechas2, on='Fechas', how='outer')
        df_filtrado = df_filtrado.sort_values(by='Fechas').reset_index(drop=True)

        return df_filtrado

    
    def imputacion_1_2(self, df, Serie_media_minutal_prom):
        """
        Esta función realiza la imputación de valores faltantes en el dataframe 'df' utilizando 
        vecinos cercanos y factor de degradación.
        
        Parámetros:
        - df: DataFrame: El dataframe que contiene los datos a imputar.
        - Serie_media_minutal_prom: Serie: La serie de valores medios por minuto utilizada para la imputación.
        
        Retorna:
        - DataFrame: El dataframe 'df' con los valores faltantes imputados.
        """
        df = df.sort_values(by='Fechas').reset_index(drop=True)
        indices_filas = []
        
        
        def prueba(df, i, count):
            """
            Esta función realiza la imputación de valores faltantes en el dataframe 'df' utilizando
            factor de degradación o remplazando por la serie media de los valores faltantes.
            
            Parámetros:
            - df: DataFrame: El dataframe que contiene los datos a imputar.
            - i: int: El índice actual en el bucle.
            - count: int: El contador de valores faltantes consecutivos.
            - valor: int: Indicador de si se debe imputar con factor de degradación o serie media.
                0: Serie media y caso contrario Factor de degradación.
            
            
            Retorna:
            - DataFrame: El dataframe 'df' con los valores faltantes imputados.
            """
            Valor_Izq = df.loc[i-count-1,"Pmax"]
            Valor_Der = df.loc[i, "Pmax"]
            
            Serie_media_Izq = Serie_media_minutal_prom[i-count-1]
            Serie_media_Der = Serie_media_minutal_prom[i]
        
            
            if Serie_media_Izq == 0 or np.isnan(Valor_Izq / Serie_media_Izq) :
                FactIni = 0
            else: 
                FactIni = Valor_Izq / Serie_media_Izq

            # Verificar si la división es NaN para Valor_Der/Serie_media_Der
            if Serie_media_Der == 0 or np.isnan(Valor_Der / Serie_media_Der)  :
                FactFin = 0
            else:
                FactFin = Valor_Der / Serie_media_Der

            Step = abs((FactFin-FactIni) / count)
            Dir = 1 if (FactFin- FactIni) >= 0 else -1
            
            FactDegra = [FactIni + Dir * k * Step for k in range(1, count+1)]

            return Serie_media_minutal_prom[i-count:i ]*np.array(FactDegra)
        
        
        count = 0
        for i in range(len(df["Fechas"])):
            
            if pd.isna(df.loc[i, "Pmax"]):
                count += 1
                
                
            elif (i != 0) and (pd.isna(df.loc[i-1,'Pmax'])) and (~pd.isna(df.loc[i,'Pmax'])):

                if count == 1:
                    df.loc[i-1, "Pmax"] = (df.loc[i-2, "Pmax"] + df.loc[i, "Pmax"]) / 2
                    count = 0
                elif count == 2:
                    df.loc[i-2, "Pmax"] = (df.loc[i-3, "Pmax"] + df.loc[i, "Pmax"]) / 2
                    df.loc[i-1, "Pmax"] = (df.loc[i-2, "Pmax"] + df.loc[i, "Pmax"]) / 2                   
                    count = 0
                elif count > 2 and count <720:
                    F = np.arange(i-count,i)
                    df.loc[F, "Pmax"] = prueba(df, i, count)
                    
                    count = 0                
                elif count >= 720:
                    F = np.arange(i-count,i)

                    df.loc[F,'Pmax'] = Serie_media_minutal_prom[i-count:i]
                    count =0
                    
        if pd.isna(df.loc[i,'Pmax']):
            if count == 1:
                df.loc[i, "Pmax"] = (df.loc[i-2, "Pmax"] + df.loc[i-1, "Pmax"]) / 2
                count = 0
            elif count == 2:
                df.loc[i-1, "Pmax"] = (df.loc[i-2, "Pmax"] + df.loc[i-3, "Pmax"]) / 2
                df.loc[i, "Pmax"] = (df.loc[i-1, "Pmax"] + df.loc[i-2, "Pmax"]) / 2                   
                count = 0
            elif count > 2 and count <720:
                F = np.arange(i-count,i)
                df.loc[F, "Pmax"] = prueba(df, i, count)
                
                count = 0                
            elif count >= 720:
                # print(i+1, " i+1")
                # print(count, " count")
                # print(i-count, " i-count")
                F = np.arange(i-count,i+1)
                # print(len(df.loc[F,'Pmax']),"F_Pmax")
                # print(len(Serie_media_minutal_prom[i-count:i+1])," Serie")
                df.loc[F,'Pmax'] = Serie_media_minutal_prom[i-count:i+1]
                count =0
                    
                
        
        # print(count, " count final ")

        return df
    




In [185]:
Analizadores = ["AnP01_","AnP03_", "AnP04_","AnP10_"]

# ruta_para_guardar = "/Users/brianrodriguez/Documents/Economia/Datos_nueva_serie"

ruta_para_guardar = "/Users/brianrodriguez/Documents/Economia/Data"
year = [2021,2022,2023]
def Crear_excel(df, nombre_analizador, año, ruta):
    with pd.ExcelWriter(f'{ruta}/{nombre_analizador}Sol_PV_{año}.xlsx', engine='openpyxl') as writer:
        df.rename(columns={'Fechas': 'datetime'}, inplace=True)
        for month, data in df.groupby(df['datetime'].dt.month):
            month_name = data['datetime'].dt.strftime('%B').iloc[0]
            data[['datetime', 'Pmax']].to_excel(writer, sheet_name=month_name, index=False)





ruta = os.getcwd()+"/Prueba_data"
ASD = Filtrado( )


In [186]:
lista = ASD.Archivos(ruta)
data = ASD.Lectura(lista)

# Para serie promedio entre 3 años #

In [182]:

for q in Analizadores:

    data_filtrado = ASD.Procesamiento(data,q)
    data_filtrado_21= data_filtrado[data_filtrado["Fechas"].dt.year == 2021].reset_index(drop=True)
    data_filtrado_22= data_filtrado[data_filtrado["Fechas"].dt.year == 2022].reset_index(drop=True)
    data_filtrado_23= data_filtrado[data_filtrado["Fechas"].dt.year == 2023].reset_index(drop=True)
    serie_21 = ASD.Series_tiempo(data_filtrado_21)
    serie_22 = ASD.Series_tiempo(data_filtrado_22)
    serie_23 = ASD.Series_tiempo(data_filtrado_23)
    V = np.array([serie_21,serie_22,serie_23])
    Serie_total = np.nanmean(V, axis=0)
    G1 = ASD.imputacion_1_2(data_filtrado_21, Serie_total)
    G2 = ASD.imputacion_1_2(data_filtrado_22, Serie_total)
    G3 = ASD.imputacion_1_2(data_filtrado_23, Serie_total)
    Crear_excel(G1,q ,2021, ruta_para_guardar)
    Crear_excel(G2,q ,2022, ruta_para_guardar)
    Crear_excel(G3,q ,2023, ruta_para_guardar)

    
        


# Para imputación Alejandro #

In [187]:

for q in Analizadores:

    data_filtrado = ASD.Procesamiento(data,q)
    data_filtrado_21= data_filtrado[data_filtrado["Fechas"].dt.year == 2021].reset_index(drop=True)
    data_filtrado_22= data_filtrado[data_filtrado["Fechas"].dt.year == 2022].reset_index(drop=True)
    data_filtrado_23= data_filtrado[data_filtrado["Fechas"].dt.year == 2023].reset_index(drop=True)
    serie_22 = ASD.Series_tiempo(data_filtrado_22)
    serie_23 = ASD.Series_tiempo(data_filtrado_23)
    serie_21 = (serie_22 + serie_23)/2
    Serie_total = np.nanmean(V, axis=0)
    G1 = ASD.imputacion_1_2(data_filtrado_21, serie_21)
    G2 = ASD.imputacion_1_2(data_filtrado_22, serie_22)
    G3 = ASD.imputacion_1_2(data_filtrado_23, serie_23)
    Crear_excel(G1,q ,2021, ruta_para_guardar)
    Crear_excel(G2,q ,2022, ruta_para_guardar)
    Crear_excel(G3,q ,2023, ruta_para_guardar)